In [1]:
import pandas as pd
import copy

In [2]:
pokemon_151 = pd.read_csv('Pokemon_151.csv', index_col = 'Unnamed: 0')
pokemon_151

,Product,Foil,Date Range,Market Price
0,151: Alakazam ex Collection,[' Market Price: '],8/5 to 8/7,$27.46
1,151: Alakazam ex Collection,[' Market Price: '],8/8 to 8/10,$26.81
2,151: Alakazam ex Collection,[' Market Price: '],8/11 to 8/13,$26.21
3,151: Alakazam ex Collection,[' Market Price: '],8/14 to 8/16,$25.41
4,151: Alakazam ex Collection,[' Market Price: '],8/17 to 8/19,$23.35
...,...,...,...,...
11785,Zubat,['Reverse Holofoil Market Price: '],10/25 to 10/27,$0.20
11786,Zubat,[' Market Price: '],10/28 to 10/30,$0.04
11787,Zubat,['Reverse Holofoil Market Price: '],10/28 to 10/30,$0.23
11788,Zubat,[' Market Price: '],10/31 to 11/2,$0.03


In [3]:
len(pokemon_151['Product'].unique())

240

In [4]:
pokemon_151['Foil'].unique()

array(["[' Market Price: ']", "['Reverse Holofoil Market Price: ']",
       "['Holofoil Market Price: ']"], dtype=object)

In [5]:
pokemon_151.replace({'Foil' : { "[' Market Price: ']" : "Normal", "[' Foil Market Price: ']" : "Foil",
                               "['Reverse Holofoil Market Price: ']" : "Reverse Holofoil", 
                               "['Holofoil Market Price: ']" : "Holofoil"}}, inplace=True)

In [6]:
pokemon_151

,Product,Foil,Date Range,Market Price
0,151: Alakazam ex Collection,Normal,8/5 to 8/7,$27.46
1,151: Alakazam ex Collection,Normal,8/8 to 8/10,$26.81
2,151: Alakazam ex Collection,Normal,8/11 to 8/13,$26.21
3,151: Alakazam ex Collection,Normal,8/14 to 8/16,$25.41
4,151: Alakazam ex Collection,Normal,8/17 to 8/19,$23.35
...,...,...,...,...
11785,Zubat,Reverse Holofoil,10/25 to 10/27,$0.20
11786,Zubat,Normal,10/28 to 10/30,$0.04
11787,Zubat,Reverse Holofoil,10/28 to 10/30,$0.23
11788,Zubat,Normal,10/31 to 11/2,$0.03


In [7]:
sum(pokemon_151['Market Price'] == "$0.00") / len(pokemon_151['Market Price'] == "$0.00")
# 49% of dataset is $0.00

0.49236641221374045

In [8]:
pokemon_151 = pokemon_151[pokemon_151['Market Price'] != '$0.00']
pokemon_151['Market Price'] = pokemon_151['Market Price'].replace({'\$':''}, regex = True)
pokemon_151['Market Price'] = pokemon_151['Market Price'].astype(float)
pokemon_151 = pokemon_151.reset_index(drop=True)
pokemon_151

C:\Users\steve\AppData\Local\Temp\ipykernel_29012\706122419.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pokemon_151['Market Price'] = pokemon_151['Market Price'].replace({'\$':''}, regex = True)
C:\Users\steve\AppData\Local\Temp\ipykernel_29012\706122419.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pokemon_151['Market Price'] = pokemon_151['Market Price'].astype(float)


,Product,Foil,Date Range,Market Price
0,151: Alakazam ex Collection,Normal,8/5 to 8/7,27.46
1,151: Alakazam ex Collection,Normal,8/8 to 8/10,26.81
2,151: Alakazam ex Collection,Normal,8/11 to 8/13,26.21
3,151: Alakazam ex Collection,Normal,8/14 to 8/16,25.41
4,151: Alakazam ex Collection,Normal,8/17 to 8/19,23.35
...,...,...,...,...
5980,Zubat,Reverse Holofoil,10/25 to 10/27,0.20
5981,Zubat,Normal,10/28 to 10/30,0.04
5982,Zubat,Reverse Holofoil,10/28 to 10/30,0.23
5983,Zubat,Normal,10/31 to 11/2,0.03


In [9]:
def past_prices(num_historical_prices, df):
    dataset = copy.deepcopy(df)
    for n in range(num_historical_prices):
        # whether product matches the product from the previous line
        match_prev_product = [dataset['Product'][i] == dataset['Product'][i+1] for i in range(len(dataset['Product'])-1)]
        # whether foil matches the foil from the previous line
        match_prev_foil = [dataset['Foil'][i] == dataset['Foil'][i+1] for i in range(len(dataset['Foil'])-1)]
        # base case, take the prev market price from the previous row
        if n == 0:
            prev_market_price = dataset['Market Price'][:-1]
        # otherwise take the latest next market price from the next row
        else:
            prev_market_price = dataset[f'Previous Market Price_{n}'][:-1]
            
        dataset = dataset[1:]
        dataset = dataset.reset_index(drop=True)
        dataset[f'Previous Market Price_{n+1}'] = prev_market_price
        prev_match = [match_prev_product[i] and match_prev_foil[i] for i in range(len(match_prev_product))]
        dataset = dataset[prev_match]
        dataset = dataset.reset_index(drop=True)
    return dataset

In [10]:
def future_prices(num_historical_prices, df):
    dataset = copy.deepcopy(df)
    for n in range(num_historical_prices):
        match_next_product = [dataset['Product'][i] == dataset['Product'][i+1] for i in range(len(dataset['Product'])-1)]
        match_next_foil = [dataset['Foil'][i] == dataset['Foil'][i+1] for i in range(len(dataset['Foil'])-1)]
        # base case, take the next market price from the next row
        if n == 0:
            next_market_price = dataset['Market Price'][1:]
        # otherwise take the latest next market price from the next row
        else:
            next_market_price = dataset[f'Next Market Price_{n}'][1:]
                    
        next_market_price = next_market_price.reset_index(drop=True)
        dataset = dataset[:-1]
        dataset = dataset.reset_index(drop=True)
        dataset[f'Next Market Price_{n+1}'] = next_market_price
        next_match = [match_next_product[i] and match_next_foil[i] for i in range(len(match_next_product))]
        dataset = dataset[next_match]
        dataset = dataset.reset_index(drop=True)
    return dataset

In [11]:
# gets the next month of market prices
pokemon_151_future = future_prices(1, pokemon_151)
# obtains the last 15 days worth of prices
pokemon_151_past_future_prices = past_prices(5, pokemon_151_future)
pokemon_151_past_future_prices

,Product,Foil,Date Range,Market Price,Next Market Price_1,Previous Market Price_1,Previous Market Price_2,Previous Market Price_3,Previous Market Price_4,Previous Market Price_5
0,151: Alakazam ex Collection,Normal,8/20 to 8/22,22.63,22.63,23.35,25.41,26.21,26.81,27.46
1,151: Alakazam ex Collection,Normal,8/23 to 8/25,22.63,22.62,22.63,23.35,25.41,26.21,26.81
2,151: Alakazam ex Collection,Normal,8/26 to 8/28,22.62,22.50,22.63,22.63,23.35,25.41,26.21
3,151: Alakazam ex Collection,Normal,8/29 to 8/31,22.50,22.42,22.62,22.63,22.63,23.35,25.41
4,151: Alakazam ex Collection,Normal,9/1 to 9/3,22.42,22.36,22.50,22.62,22.63,22.63,23.35
...,...,...,...,...,...,...,...,...,...,...
853,Zapdos ex - 202/165,Holofoil,10/16 to 10/18,42.84,41.67,43.75,46.11,43.29,39.52,43.81
854,Zapdos ex - 202/165,Holofoil,10/19 to 10/21,41.67,42.31,42.84,43.75,46.11,43.29,39.52
855,Zapdos ex - 202/165,Holofoil,10/22 to 10/24,42.31,42.06,41.67,42.84,43.75,46.11,43.29
856,Zapdos ex - 202/165,Holofoil,10/25 to 10/27,42.06,41.97,42.31,41.67,42.84,43.75,46.11


# Data Split

In [13]:
pokemon_151_past_future_prices['Date Range'].unique()

array(['8/20 to 8/22', '8/23 to 8/25', '8/26 to 8/28', '8/29 to 8/31',
       '9/1 to 9/3', '9/4 to 9/6', '9/7 to 9/9', '9/10 to 9/12',
       '9/13 to 9/15', '9/16 to 9/18', '9/19 to 9/21', '9/22 to 9/24',
       '9/25 to 9/27', '9/28 to 9/30', '10/1 to 10/3', '10/4 to 10/6',
       '10/7 to 10/9', '10/10 to 10/12', '10/13 to 10/15',
       '10/16 to 10/18', '10/19 to 10/21', '10/22 to 10/24',
       '10/25 to 10/27', '10/28 to 10/30'], dtype=object)

In [14]:
test = pokemon_151_past_future_prices.loc[pokemon_151_past_future_prices['Date Range'].isin(['10/16 to 10/18', '10/19 to 10/21', '10/22 to 10/24',
       '10/25 to 10/27', '10/28 to 10/30'])]
train = pokemon_151_past_future_prices.loc[pokemon_151_past_future_prices['Date Range'].isin(['8/20 to 8/22', '8/23 to 8/25', '8/26 to 8/28', '8/29 to 8/31',
       '9/1 to 9/3', '9/4 to 9/6', '9/7 to 9/9', '9/10 to 9/12',
       '9/13 to 9/15', '9/16 to 9/18', '9/19 to 9/21', '9/22 to 9/24',
       '9/25 to 9/27', '9/28 to 9/30', '10/1 to 10/3', '10/4 to 10/6',
       '10/7 to 10/9', '10/10 to 10/12', '10/13 to 10/15'])]
test

,Product,Foil,Date Range,Market Price,Next Market Price_1,Previous Market Price_1,Previous Market Price_2,Previous Market Price_3,Previous Market Price_4,Previous Market Price_5
19,151: Alakazam ex Collection,Normal,10/16 to 10/18,24.40,25.65,22.82,25.33,24.35,26.56,31.60
20,151: Alakazam ex Collection,Normal,10/19 to 10/21,25.65,26.64,24.40,22.82,25.33,24.35,26.56
21,151: Alakazam ex Collection,Normal,10/22 to 10/24,26.64,25.79,25.65,24.40,22.82,25.33,24.35
22,151: Alakazam ex Collection,Normal,10/25 to 10/27,25.79,25.26,26.64,25.65,24.40,22.82,25.33
23,151: Alakazam ex Collection,Normal,10/28 to 10/30,25.26,25.40,25.79,26.64,25.65,24.40,22.82
...,...,...,...,...,...,...,...,...,...,...
853,Zapdos ex - 202/165,Holofoil,10/16 to 10/18,42.84,41.67,43.75,46.11,43.29,39.52,43.81
854,Zapdos ex - 202/165,Holofoil,10/19 to 10/21,41.67,42.31,42.84,43.75,46.11,43.29,39.52
855,Zapdos ex - 202/165,Holofoil,10/22 to 10/24,42.31,42.06,41.67,42.84,43.75,46.11,43.29
856,Zapdos ex - 202/165,Holofoil,10/25 to 10/27,42.06,41.97,42.31,41.67,42.84,43.75,46.11


In [15]:
X_test = test[[ 'Foil', 'Previous Market Price_1', 'Previous Market Price_2', 'Previous Market Price_3',
              'Previous Market Price_4', 'Previous Market Price_5', 'Market Price']]
y_test = test['Next Market Price_1'].astype(float)
X_train = train[[ 'Foil', 'Previous Market Price_1', 'Previous Market Price_2', 'Previous Market Price_3',
              'Previous Market Price_4', 'Previous Market Price_5', 'Market Price']]
y_train = train['Next Market Price_1'].astype(float)

In [16]:
X_test = pd.get_dummies(X_test, columns = ['Foil'])
X_train = pd.get_dummies(X_train, columns = ['Foil'])